In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/MyDrive/AI/experiment/DrivingHelper/

/content/gdrive/MyDrive/AI/experiment/DrivingHelper


In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

import tensorflow as tf

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [40]:
#### NORMALIZING DATA

from sklearn import preprocessing

beginner_path = 'beginner_expert_processedData/beginner/beginner_'
expert_path = 'beginner_expert_processedData/expert/expert_'
left_column = [
#'Time',
    'Distance','GPS Latitude','GPS Longitude','Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
'Damper Velocity (Calc) RR','Corr Dist','Corr Dist (Unstretched)','Corr Speed','Brake Pos',
'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
'Self Align Torque RL','Self Align Torque RR','Session Time Left','Steering Angle','Suspension Travel FL','Suspension Travel FR',
'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
'CG Distance','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration']

curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]
curve_num = 0
sequences = list()
final_df = pd.DataFrame()
scaler = preprocessing.StandardScaler()

for i in range(1,36):
    file_path = beginner_path + str(i) + '_new2.csv'
    #print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    for j in range(0, 6):
      tmp_df = df[(df['Distance'] >= curveList[j][0]) & (df['Distance'] <= curveList[j][1])]
      #print(tmp_df)
      tmp_df = tmp_df.loc[:,left_column]
      tmp_df['curve_num'] = j
      final_df = final_df.append(tmp_df, ignore_index=True)
    values = tmp_df.values
    #val_scaled = scaler.fit_transform(values)
    sequences.append(values)


    
for i in range(1,20):
    file_path = expert_path + str(i) + '_new2.csv'
    #print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    for j in range(0, 6):
      tmp_df = df[(df['Distance'] >= curveList[j][0]) & (df['Distance'] <= curveList[j][1])]
      tmp_df = tmp_df.loc[:,left_column]
      tmp_df['curve_num'] = j
      final_df = final_df.append(tmp_df, ignore_index=True)
    values = tmp_df.values
    #val_scaled = scaler.fit_transform(values)
    sequences.append(values)
    

In [9]:
#### modified!!!

beginner_path = 'beginner_expert_processedData/beginner/beginner_'
expert_path = 'beginner_expert_processedData/expert/expert_'
left_column = [
#'Time',
    'Distance','GPS Latitude','GPS Longitude','Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
'Damper Velocity (Calc) RR','Corr Dist','Corr Dist (Unstretched)','Corr Speed','Brake Pos',
'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
'Self Align Torque RL','Self Align Torque RR','Session Time Left','Steering Angle','Suspension Travel FL','Suspension Travel FR',
'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
'CG Distance','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration']

curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]
curve_num = 0
sequences = list()
final_df = pd.DataFrame()
for i in range(1,36):
    file_path = beginner_path + str(i) + '_new2.csv'
    #print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    for j in range(0, 6):
      tmp_df = df[(df['Distance'] >= curveList[j][0]) & (df['Distance'] <= curveList[j][1])]
      #print(tmp_df)
      tmp_df = tmp_df.loc[:,left_column]
      tmp_df['curve_num'] = j
      final_df = final_df.append(tmp_df, ignore_index=True)
    values = tmp_df.values
    sequences.append(values)

    
for i in range(1,20):
    file_path = expert_path + str(i) + '_new2.csv'
    #print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    for j in range(0, 6):
      tmp_df = df[(df['Distance'] >= curveList[j][0]) & (df['Distance'] <= curveList[j][1])]
      tmp_df = tmp_df.loc[:,left_column]
      tmp_df['curve_num'] = j
      final_df = final_df.append(tmp_df, ignore_index=True)
    values = tmp_df.values
    sequences.append(values)
    

In [41]:
print(len(sequences))

54


In [42]:
print(len(sequences))
print(sequences[1].shape)

54
(148, 103)


In [43]:
targets = pd.read_csv('target.csv')
targets = targets.values[:,1]

In [17]:
targets

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [20]:
groups

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 3, 3, 3, 3])

In [44]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
    
print(len_sequences)
pd.Series(len_sequences).describe()

[125, 148, 151, 146, 129, 147, 145, 131, 145, 146, 154, 150, 194, 156, 183, 136, 142, 121, 494, 139, 141, 130, 148, 149, 133, 141, 138, 157, 115, 119, 171, 147, 184, 167, 237, 123, 132, 124, 125, 117, 126, 129, 119, 126, 122, 133, 126, 128, 125, 138, 117, 127, 133, 131]


count     54.000000
mean     147.407407
std       52.783116
min      115.000000
25%      126.000000
50%      137.000000
75%      148.000000
max      494.000000
dtype: float64

In [45]:
#Padding the sequence with the values in last row to max length
to_pad = 494
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(103, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from keras.preprocessing import sequence
seq_len = 115
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [46]:
groups = pd.read_csv('group.csv', header=0)
groups = groups.values[:,1]

In [47]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==1)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==2]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==1)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==2]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

train

[array([[1212.7       ,   44.05425462,  -78.68331907, ...,  -10.80305127,
          -14.40406911,    5.        ],
        [1214.7       ,   44.05426388,  -78.68333499, ...,   -7.20203456,
          -21.6061031 ,    5.        ],
        [1216.6       ,   44.05427305,  -78.68335045, ...,  -32.40915408,
          -61.2172906 ,    5.        ],
        ...,
        [1412.        ,   44.05574569,  -78.68386963, ...,  -10.80305127,
           25.20712009,    5.        ],
        [1414.        ,   44.0557635 ,  -78.68386559, ...,  -18.00508525,
           28.80813766,    5.        ],
        [1416.        ,   44.05578014,  -78.68386163, ...,   25.20711981,
           43.21220563,    5.        ]]),
 array([[ 1.21380000e+03,  4.40544472e+01, -7.86836110e+01, ...,
          1.08030513e+01, -3.60101716e+01,  5.00000000e+00],
        [ 1.21530000e+03,  4.40544553e+01, -7.86836224e+01, ...,
         -7.20203427e+00, -6.12172906e+01,  5.00000000e+00],
        [ 1.21680000e+03,  4.40544636e+01, -7.868

In [48]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
validation_target = np.array(validation_target)
test_target = np.array(test_target)

print(train.shape)
print(train_target.shape)

(34, 115, 103)
(34,)


In [18]:
!pip install keras-tuner
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from tensorflow import keras

     |████████████████████████████████| 71kB 5.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=80719778d3d585d75dd617c71fce1a7f16a89b16f92ba060ecb3fccfd9afea45
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=7e2b92334fa64963787f9008218e120b5bf7deb7be2152b82800290fa4c526f1
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [49]:
from tensorflow.keras.models import Sequential
#alpha?
def build_model(hp):
  model = Sequential()
  model.add(LSTM(units=hp.Int('units', min_value=32, max_value=1024, default=256, step=2), input_shape=(seq_len, 103)))
  model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=Adam(
      hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  ), metrics=['accuracy'])
  return model

tuner=RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,
    executions_per_trial=2,
    directory='my_dir',
    project_name='driverLSTM!'
)

callback= tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tuner.search(train, train_target, epochs=100, validation_data=(validation,validation_target), callbacks=[callback])
tuner.results_summary()

Trial 30 Complete [00h 00m 07s]
val_accuracy: 0.800000011920929

Best val_accuracy So Far: 0.800000011920929
Total elapsed time: 00h 03m 29s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in my_dir/driverLSTM!
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 354
dropout: 0.4
learning_rate: 0.001
Score: 0.800000011920929
Trial summary
Hyperparameters:
units: 654
dropout: 0.2
learning_rate: 0.001
Score: 0.800000011920929
Trial summary
Hyperparameters:
units: 796
dropout: 0.1
learning_rate: 0.0001
Score: 0.75
Trial summary
Hyperparameters:
units: 390
dropout: 0.05
learning_rate: 0.0001
Score: 0.75
Trial summary
Hyperparameters:
units: 344
dropout: 0.05
learning_rate: 0.0001
Score: 0.75
Trial summary
Hyperparameters:
units: 846
dropout: 0.0
learning_rate: 0.0001
Score: 0.75
Trial summary
Hyperparameters:
units: 870
dropout: 0.1
learning_rate: 0.0001
Score: 0.75
Trial summary
Hyperparameters:
units: 744
dropout: 0.25
learning_rate: 0.0001
Score: 0.7000000178813934
Trial summary
Hyperparameters:
units: 152
dropout: 0.1
learning_rate: 0.001
Score: 0.7000000178813934
Trial summary
Hyperparameter

In [50]:
model = Sequential()
model.add(LSTM(354, input_shape=(seq_len, 103)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [51]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 0.7766 - accuracy: 0.5294 - val_loss: 0.6634 - val_accuracy: 0.7000


Epoch 2/200
1/1 [==============================] - 0s 48ms/step - loss: 0.5980 - accuracy: 0.7059 - val_loss: 0.6630 - val_accuracy: 0.6000


Epoch 3/200
1/1 [==============================] - 0s 51ms/step - loss: 0.5888 - accuracy: 0.5882 - val_loss: 0.6252 - val_accuracy: 0.6000


Epoch 4/200
1/1 [==============================] - 0s 48ms/step - loss: 0.5953 - accuracy: 0.7059 - val_loss: 0.6558 - val_accuracy: 0.6000


Epoch 5/200
1/1 [==============================] - 0s 58ms/step - loss: 0.5435 - accuracy: 0.7647 - val_loss: 0.6584 - val_accuracy: 0.6000


...

Epoch 196/200
1/1 [==============================] - 0s 45ms/step - loss: 0.3881 - accuracy: 0.7647 - val_loss: 0.4552 - val_accuracy: 0.8000


Epoch 197/200
1/1 [==============================] - 0s 47ms/step - loss: 0.3448 - accuracy: 0.8235 - val_loss: 0.4535 - val_accuracy: 0.8000


Epoch 198/200
1/1 [==============================] - 0s 52ms/step - loss: 0.3700 - accuracy: 0.8235 - val_loss: 0.4495 - val_accuracy: 0.8000


Epoch 199/200
1/1 [==============================] - 0s 45ms/step - loss: 0.3655 - accuracy: 0.8529 - val_loss: 0.4506 - val_accuracy: 0.8000


Epoch 200/200
1/1 [==============================] - 0s 46ms/step - loss: 0.4007 - accuracy: 0.8235 - val_loss: 0.4500 - val_accuracy: 0.8000


In [52]:
model.evaluate(test,test_target)

1/1 [==============================] - 0s 25ms/step - loss: 0.3017 - accuracy: 0.9000


[0.3016512989997864, 0.8999999761581421]

In [ ]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))

In [67]:
model.evaluate(test,test_target)

1/1 [==============================] - 0s 18ms/step - loss: 0.3332 - accuracy: 0.9000


[0.33322328329086304, 0.8999999761581421]